In [1]:
import pandas as pd
import sys
import os
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")


In [67]:
data_raw = pd.read_csv('data/leagues/model_dataset_raw.csv',
                       sep=';', index_col=0, decimal=',')
data_raw['Date'] = pd.to_datetime(data_raw['Date'])
data_raw.index = data_raw.index+"__TEAM_"+data_raw.Team
data_raw = data_raw[~data_raw['FTR'].isna()]
data_raw.Target = data_raw.FTG > data_raw.FTOG


Für Regressor

In [68]:
# data_raw["Target"][data_raw.FTG == data_raw.FTOG]=0
# data_raw["Target"][data_raw.FTG > data_raw.FTOG]=1
# data_raw["Target"][data_raw.FTG < data_raw.FTOG]=-1


In [2]:
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score, roc_auc_score,
                             roc_curve)
import numpy as np
import shap


def set_dummies_div(df, cat, divs=[]):
    if cat in df.columns:
        if divs == []:
            divs = list(set(df[cat]))
        for d in divs:
            df[d] = [1 if ele == d else 0 for ele in df[cat]]
        df = df.drop(cat, axis=1)
    return df


def evaluate_model(model, y_test, X_test, show=False,threshold=''):
    if threshold == '':
        y_pred = model.predict(X_test)
    else:
       y_pred = model.predict_proba(X_test)[:,1]>threshold

    modellguete = dict()
    modellguete["roc_auc_score"] = np.round(roc_auc_score(y_test, y_pred,), 3)
    modellguete["f1_score"] = np.round(f1_score(y_test, y_pred), 3)
    modellguete["accuracy_score"] = np.round(accuracy_score(y_test, y_pred), 3)
    modellguete["precision_score"] = np.round(
        precision_score(y_test, y_pred), 3)
    modellguete["recall_score"] = np.round(recall_score(y_test, y_pred), 3)

    print('AUC Score : ', modellguete["roc_auc_score"])
    print('F1 Score : ', modellguete["f1_score"])
    print('Accuracy Score : ', modellguete["accuracy_score"])
    print('Precision Score : ', modellguete["precision_score"])
    print('Recall Score : ', modellguete["recall_score"])

    # Model Accuracy, how often is the classifier correct?
    print('Confusion Matrix : \n' +
          str(np.round(confusion_matrix(y_test, y_pred), 3)))
    if False:
        try:
            print('-'*100)
            print('Feature Importance',)
            feature_imp = pd.Series(
                model.feature_importances_, index=X_test.columns).sort_values(ascending=False)[0:15]
            feature_imp.to_json()
            print('Important')
            print(feature_imp)
            feature_not_imp = pd.Series(
                model.feature_importances_, index=X_test.columns).sort_values(ascending=True)[0:15]
            print('Not Important')
            print(feature_not_imp)
        except:
            print('Fehler bei Feature Importance')
            pass

        try:
            print('-'*100)
            print('Shap Values',)
            rf_shap_values = shap.TreeExplainer(model).shap_values(
                X_test.sample(100, random_state=42))
            shap_df = pd.DataFrame(rf_shap_values[0], columns=X_test.columns)
            print(np.abs(shap_df).mean().sort_values(ascending=False))
        except:
            pass

        try:
            X_test.corr().round(2)[X_test.corr().round(2) > 0.3].to_csv(
                'C:/Users/Robert/Documents/corr.csv', sep=';', decimal=',', index=True)
        except:
            pass

    print('-'*100)
    return modellguete


In [3]:
data = data_raw[['FTR', 'VC', 'VCO', 'B365',
                 'B365O', 'BW',  'BWO', 'Date', 'Target','atHome']]


NameError: name 'data_raw' is not defined

# PreProcess

In [71]:
data = data[data.FTR != "D"]

## Monate verarbeiten

In [72]:
data['month'] = data.Date.dt.month
data.month = [i-6 if i > 6 else i+6 for i in data.month]


### Trenne Trainings-und Testdaten (data) und Evaluierung (X_eval)

In [73]:
import datetime

datum_1_1_2020 = datetime.datetime(2022, 6, 1)
evaluierung = data.Date >= datum_1_1_2020
X_eval = data[evaluierung].drop(['Target'], axis=1)
y_eval = data[evaluierung]['Target']

Wähle Training und Testdaten

In [74]:

training = data.Date < datum_1_1_2020
data = data[training]
data = data.drop(['Date'],axis=1)

In [75]:
#data = data[(data.B365>1.9) & (data.B365<2.5)]

### Verarbeitung Training und Test

Entferne Unentschieden im Training

In [76]:
data = data[data.FTR != "D"]
data = data.drop('FTR', axis=1)


 Nur Trainigsligen für Training Test

In [77]:
if False:
    data = data[[True if i in ['D1'] else False for i in data.Div]]
    data = data.drop('Div', axis=1)


Trenne Daten in Einzeldatensätze auf

In [78]:
from sklearn.model_selection import train_test_split

y = data['Target']
X = data.drop(['Target'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, stratify=y)


## Dummy Encoding

In [79]:
if 'Div' in X_train.columns:
    divs = list(set(X_train['Div']))
    X_train = set_dummies_div(df=X_train, cat='Div', divs=divs)
    X_test = set_dummies_div(df=X_test, cat='Div', divs=divs)
    X_eval = set_dummies_div(df=X_eval, cat='Div', divs=divs)


In [80]:
if False:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler = scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_eval = scaler.transform(X_eval)


In [81]:
if False:
    from sklearn.neighbors import LocalOutlierFactor
    clf = LocalOutlierFactor(n_neighbors=10)
    outlier_bool = clf.fit_predict(X_train) > 0
    X_train = X_train[outlier_bool]
    y_train = y_train[outlier_bool]


In [87]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV  
from xgboost import XGBClassifier, XGBRegressor
# model = RandomForestClassifier(
#     n_estimators=200, random_state=0, verbose=1, min_samples_leaf=5, min_samples_split=10)
model = XGBClassifier(n_estimators=200,random_state = 0,verbose=1, min_samples_leaf= 5, min_samples_split = 10)
parameters = {
	'selector__threshold': [0],
	'classifier__n_neighbors': [1],
	'classifier__max_depth': [5],
	'classifier__learning_rate': [0.01],
	'classifier__n_estimators': [100,]
}
 
 
model = GridSearchCV(XGBClassifier(), parameters,cv=10,scoring = 'precision').fit(X_train, y_train)
#model = gridsearch(model, X_train, y_train)
#model = XGBRegressor()
model.fit(X=X_train, y=y_train)


[22:05:22] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { classifier__learning_rate, classifier__max_depth, classifier__n_estimators, classifier__n_neighbors, selector__threshold } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:05:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { classifier__learning_rate, classifier__max_depth, classifier__n_estimators, classifier__n_neighbors, selector__threshold } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:

GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_job...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

# Ergebnisse

In [92]:
erg_test = evaluate_model(model, y_test, X_test, show=False,threshold=0.7)


ValueError: y should be a 1d array, got an array of shape (23810, 2) instead.

In [85]:
X_eval = X_eval.drop(['Date','FTR'],axis=1)

In [89]:
erg_test = evaluate_model(model, y_eval, X_eval, show=False)


AUC Score :  0.684
F1 Score :  0.68
Accuracy Score :  0.684
Precision Score :  0.687
Recall Score :  0.673
Confusion Matrix : 
[[444 196]
 [209 431]]
----------------------------------------------------------------------------------------------------


# Einzelanalyse

 1    640
-1    640
 0    462
Name: Target, dtype: int64

In [65]:
y_pred = model.predict(X_eval)
y_pred[abs(y_pred)<0.1]=0
y_pred[y_pred<0]=-1
y_pred[y_pred>0]=1
(y_pred==y_eval).sum()/len(y_pred)
pd.Series(y_pred).value_counts()
(y_pred==y_eval).sum()/len(y_pred)


0.47876004592422505

0.40298507462686567

In [23]:
join_df = pd.DataFrame(data_raw[data_raw['Div'] == 'E2']['Div'])
X_eval_league = X_eval.join(join_df, how='inner').drop('Div', axis=1)
y_eval_league = pd.DataFrame(y_eval).join(join_df, how='inner').Target

evaluate_model(model, y_eval_league, X_eval_league, show=False)


AUC Score :  0.673
F1 Score :  0.601
Accuracy Score :  0.661
Precision Score :  0.518
Recall Score :  0.717
Confusion Matrix : 
[[68 40]
 [17 43]]
----------------------------------------------------------------------------------------------------


{'roc_auc_score': 0.673,
 'f1_score': 0.601,
 'accuracy_score': 0.661,
 'precision_score': 0.518,
 'recall_score': 0.717}

In [40]:
sum(~y_eval_league)

30

In [29]:
my_list = {}
ligen = list(set(data_raw.Div))
for d in ligen:
    print(d)
    try:
        div_bool = (data_raw['Div'] == d)
        draw_bool =  (data_raw['FTR'] == 'D')
        #sel_bool = (data_raw.B365 < 2)
        join_df = pd.DataFrame(data_raw[div_bool&draw_bool]['Div'])
        
        X_eval_league = X_eval.join(join_df, how='inner').drop('Div', axis=1)
        y_eval_league = pd.DataFrame(y_eval).join(join_df, how='inner').Target

        erg = evaluate_model(model, y_eval_league, X_eval_league, show=True)
        erg['anzahl_spiele'] = len(y_eval_league)
        my_list[d] = erg
    except:
        print('Fehler')
        pass
erg_liga = pd.DataFrame(my_list).T
erg_liga.sort_values("precision_score", ascending=False)

SP2
Fehler
E3
Fehler
F2
Fehler
E1
Fehler
F1
Fehler
SC0
Fehler
E0
Fehler
N1
Fehler
T1
Fehler
E2
Fehler
I1
Fehler
D1
Fehler
P1
Fehler
B1
Fehler
SP1
Fehler
G1
Fehler
I2
Fehler
D2
Fehler


KeyError: 'precision_score'

In [ ]:

data_raw[(data_raw.Target == True) & (
    data_raw.Div == 'I1')].Team.value_counts()


In [ ]:
y_pred = model.predict(X_eval)
y_pred = pd.DataFrame(y_pred, index=X_eval.index, columns=["Pred"])


In [ ]:
ergebnisse = pd.concat([y_eval, y_pred], axis=1)
ergebnisse = pd.concat([ergebnisse, data_raw[['Div', 'FTR', 'Team']]], axis=1)
ergebnisse = ergebnisse[~ergebnisse.Target.isna()]
ergebnisse


In [ ]:
falsche_Prognose = ~(ergebnisse.Target == ergebnisse.Pred)
ergebnisse[falsche_Prognose].Div.value_counts()/ergebnisse.Div.value_counts()


In [ ]:
falsche_teams = pd.DataFrame(
    ergebnisse[falsche_Prognose].Team.value_counts()).T


In [ ]:
falsche_teams.to_csv("test.csv")


In [ ]:
data_raw.loc[ergebnisse[falsche_Prognose]
             .index].loc['2016-02-07__Hoffenheim__Darmstadt__TEAM_Darmstadt']


In [ ]:
data_raw.corr().round(2).Target.sort_values()
